# Def Chain

In [ ]:
import matplotlib.pyplot as plt
import multiprocessing
from matplotlib.patches import FancyArrowPatch

import sys

In [1]:
class OutputResDefChain:
    motiDict = None    
    type2API = None    
    winRes2poli = None 
    
    fam_name = None
    path_comMotif_dir = None
    
    
    def __init__(self, claMotiAnaly, claStageMatrix):
        self.motiDict = claMotiAnaly.getMotiDict()
        self.type2API = claMotiAnaly.getType2API()
        self.winRes2poli = claMotiAnaly.getWinRes2point()
        
        famPath = claStageMatrix.getFamPath()
        self.fam_name = famPath.split('/')[-1]
        self.path_comMotif_dir = famPath +'/'+self.fam_name +'_final common motif/'        
        if not os.path.isdir(self.path_comMotif_dir): 
            os.makedirs(self.path_comMotif_dir)       
    
        print self.winRes2poli.keys()
        print self.winRes2poli
        while True:
            self.__resDefChain()
            
    
    #===private function
    # 畫 path於 timeline-category graph上，幾個變數就有幾個
    # 選擇要輸出的Def chain圖
    # input: resource
    # output: Def chain path on API category graph
    def __resDefChain(self): 
        winRes2poli_dict = self.winRes2poli
        motif_dict = self.motiDict
        type2API_dict = self.type2API
        
        var = raw_input('Input variables extracted above:')

        plt.figure(num=None, figsize=(18, 6), dpi=80, facecolor='w', edgecolor='k')
        plt.subplot(4,1,1)
        ax1 = plt.subplot2grid((4,1), (0,0), rowspan=3)
        plt.title('API category graph\n'+var, fontsize=16)
        plt.ylabel('Category')
        plt.xlim(0,len(motif_dict)+2)
        plt.ylim(0,6)
        for i, k in enumerate(sorted(motif_dict, key=lambda k: int(k[1:k.index('_')]))):
            for t,a in motif_dict[k]:
                api = a[:a.index('#')]
                category = [cat for cat in type2API_dict for ain in type2API_dict[cat] if ain == api][0]
                if category == 'Library':
                    plt.plot(i+1, 1, c='r',marker='o')
                elif category == 'Process':
                    plt.plot(i+1, 2, c='y',marker='o')
                elif category == 'File':
                    plt.plot(i+1, 3, c='b',marker='o')
                elif category == 'Registry':
                    plt.plot(i+1, 4, c='k',marker='o')
                elif category == 'Network':
                    plt.plot(i+1, 5, c='g',marker='o')

        plt.yticks(range(1,6), ['Library', 'Process', 'File', 'Registry', 'Network'])  
        plt.tick_params(
            axis='x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom='off',      # ticks along the bottom edge are off
            top='off',         # ticks along the top edge are off
            labelbottom='off') 
        # for ii,(x,y) in enumerate(var2plist_dict[var]): #=====10/5  sorted(resFreq_dist.items(), key=lambda (k,v): v[0])
        winRes2poli_dict[var] = sorted(winRes2poli_dict[var], key=lambda (k,v): k)
        for ii,(x,y) in enumerate(winRes2poli_dict[var]):
            #print ii, x, y 
#             #放大經過路徑
#             if y == 1:
#                 plt.plot(x,y,c='r',marker='o',ms=20)
#             elif y == 2:
#                 plt.plot(x,y,c='y',marker='o',ms=20)
#             elif y == 3:
#                 plt.plot(x,y,c='b',marker='o',ms=20)
#             elif y == 4:
#                 plt.plot(x,y,c='k',marker='o',ms=20)
#             elif y == 5:
#                 plt.plot(x,y,c='g',marker='o',ms=20)
            if ii+1 < len(winRes2poli_dict[var]):
                t = FancyArrowPatch(posA=(x,y),posB = winRes2poli_dict[var][ii+1],
                                        connectionstyle = 'arc3 ,rad=0.1',
                                        arrowstyle = 'simple',
                                        mutation_scale = 30)
                ax1.add_patch(t)

        #畫每stage的長度為一長條圖
        ax2 = plt.subplot2grid((4,1), (3,0))
        plt.yticks(range(0,
                        max([len(motif_dict[m]) for m in sorted(motif_dict, key=lambda k: int(k[1:k.index('_')]))])+5,
                        (max([len(motif_dict[m]) for m in sorted(motif_dict, key=lambda k: int(k[1:k.index('_')]))])+5)/3))
        plt.xlim(0,len(motif_dict)+2)
        plt.ylim(0, max([len(motif_dict[m]) for m in sorted(motif_dict, key=lambda k: int(k[1:k.index('_')]))])+5)
        plt.vlines([range(1,len(motif_dict)+1)], 
                   [0]*len(motif_dict), 
                   [len(motif_dict[m]) for m in sorted(motif_dict, key=lambda k: int(k[1:k.index('_')]))])
        for i, m in enumerate(sorted(motif_dict, key=lambda k: int(k[1:k.index('_')]))):
            plt.annotate(str(len(motif_dict[m])), xy=(i+1, len(motif_dict[m])), size='small')

        #spacing

        plt.xlabel('Timeline (ordered in stage index)')
        plt.ylabel('API numbers')
        plt.tight_layout()
        # plt.savefig('data/final common motif in Firseria/Timline - category graph'+' ('+var+') '+'.png', dpi=300)
        plt.savefig(self.path_comMotif_dir+ self.fam_name + '_defChain ('+var+') '+'.png', dpi=300)#----------
        plt.clf()

        print var + ' done!'
